In [4]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

import pandas as pd
import scorecardpy as sc

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

In [5]:
def calcular_pesos(modelo):
    coef_model = pd.DataFrame(modelo.tvalues,columns=['t_value'])
    coef_model['t_value2'] = np.power(coef_model['t_value'],2)
    coef_model['total'] = sum(coef_model['t_value2'])
    coef_model['part'] = coef_model['t_value2'] / coef_model['total']
    coef_model['pesos'] = coef_model['part'] * 100
    return coef_model['pesos'].sort_values(ascending=True)

In [6]:
x_train = pd.read_csv("X_train.csv")
x_valid = pd.read_csv("X_valid.csv")

y_train = pd.read_csv("y_train.csv")
y_valid = pd.read_csv("y_valid.csv")

x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((54315, 59), (9586, 59), (54315, 1), (9586, 1))

In [7]:
def corr_detail(df_corr, min_value=0.85, show_detail=True, col_target='target'):
    
    corr_taret = df_corr[col_target].sort_values(ascending=False).apply(lambda _: abs(_)).sort_values(ascending=False)
    
    from itertools import combinations
    def add_dicc(key, val, num, dicc):
        if key in dicc:
            dicc[key].append((val, num))
        else:
            dicc[key] = [(val, num)]
        
    set_unique, list_detected = set(), []
    dicc_detected = {}
    

    for cols in combinations(list(df_corr.columns), 2):
        corr_columns = np.fabs(df_corr[cols[0]][cols[1]])
        
        if corr_columns >= min_value:
            set_unique.update(cols)
            list_detected.extend(list(cols))
            add_dicc(cols[0], cols[1], corr_columns, dicc_detected)
            add_dicc(cols[1], cols[0], corr_columns, dicc_detected)        
            
    print("ANALISIS CORRELACION ENTRE COLUMNAS :")
    columnas_aparece = []
    cols_del = []
    for colc, cant in sorted(
        [(col, list_detected.count(col)) for col in set_unique], key=lambda _: _[1], reverse=True):
        
        if colc not in columnas_aparece:
            print("/"*30)
            print("-> ", colc, ' :::: ',cant, ' ///// ', corr_taret[colc])
            columnas_aparece.append(colc)
            if show_detail:
                for _ in dicc_detected[colc]:
                    print(" "*10, *_, '///// ',  corr_taret[_[0]])
                    #columnas_aparece.append(_[0])
                    
                    if corr_taret[_[0]] <= corr_taret[colc]:
                        cols_del.append(_[0])
                    else:
                        cols_del.append(colc)
                    
    del dicc_detected
    del list_detected
    del set_unique
    return list(set(cols_del))

### >> Modleo TOP IV

In [9]:
cols_aporte = [col for col in x_train.columns if col.lower() not in ['target', 'incumplimiento']]
print(cols_aporte)

['Empleado', 'Tiempo_empleo', 'Saldo_cuenta', 'Valor_prestamo', 'Cuentas_otros', 'Autocontrol', 'Impulsividad', 'Confianza', 'cuentas_otros_x_valor_prestamo', 'cuentas_otros_x_valor_prestamo_entre_tiempo_empleo', 'cuentas_otros_x_saldo_cuenta', 'cuentas_otros_x_saldo_cuenta_entre_tiempo_empleo', 'cuentas_otros_x_tiempo_empleo', 'cuentas_otros_x_autocontrol', 'cuentas_otros_x_autocontrol_entre_tiempo_empleo', 'cuentas_otros_x_impulsividad', 'cuentas_otros_x_impulsividad_entre_tiempo_empleo', 'cuentas_otros_x_confianza', 'cuentas_otros_x_confianza_entre_tiempo_empleo', 'valor_prestamo_x_saldo_cuenta', 'valor_prestamo_x_saldo_cuenta_entre_cuentas_otros', 'valor_prestamo_x_saldo_cuenta_entre_tiempo_empleo', 'valor_prestamo_x_tiempo_empleo', 'valor_prestamo_x_tiempo_empleo_entre_cuentas_otros', 'valor_prestamo_x_autocontrol', 'valor_prestamo_x_autocontrol_entre_cuentas_otros', 'valor_prestamo_x_autocontrol_entre_tiempo_empleo', 'valor_prestamo_x_impulsividad', 'valor_prestamo_x_impulsividad

In [ ]:
x_train['target'] = y_train.values
matriz_corr = x_train[cols_aporte + ['target']].corr(method='pearson')

matriz_corr.style.background_gradient(
    cmap='coolwarm', axis=None
).set_precision(5)

In [ ]:
pd.DataFrame(matriz_corr['target'].abs().sort_values(ascending=False)).style.background_gradient(
    cmap='coolwarm', axis=None
).set_precision(5)

In [ ]:
matriz_corr = x_train[cols_aporte + ['target']].corr(method='spearman')

matriz_corr.style.background_gradient(
    cmap='coolwarm', axis=None
).set_precision(5)

In [ ]:
pd.DataFrame(matriz_corr['target'].abs().sort_values(ascending=False)).style.background_gradient(
    cmap='coolwarm', axis=None
).set_precision(5)

In [ ]:
#if 'target' in x_train.columns:
#    del x_train['target']

In [ ]:
corr_detail(matriz_corr, min_value=0.85, show_detail=True, col_target='target')

In [ ]:
for col in ['autocontrol_x_confianza_entre_tiempo_empleo', 'autocontrol_x_impulsividad_entre_tiempo_empleo',
            'valor_prestamo_x_impulsividad_entre_tiempo_empleo', 'saldo_cuenta_x_autocontrol']:
    del x_train[col]
    del x_valid[col]
    cols_aporte.remove(col)

In [ ]:
import statsmodels.api as sm

help(sm.Logit)

In [ ]:
import statsmodels.api as sm

logit_model_2 = sm.Logit(y_train , x_train[cols_aporte])
result_2 = logit_model_2.fit()

In [ ]:
result_2.summary2()

In [ ]:
import numpy as np
#prediciendo en base al modelo 
y_pred_prob_2_train = result_2.predict(x_train[cols_aporte])
y_pred_class_2_train = np.where(y_pred_prob_2_train < 0.5, 0 , 1)

y_pred_prob_2_valid = result_2.predict(x_valid[cols_aporte])
y_pred_class_2_valid = np.where(y_pred_prob_2_valid < 0.5, 0 , 1)

In [ ]:
y_pred_prob_2_train.hist()

In [ ]:
y_pred_prob_2_valid.hist()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

print("/"*25, 'TRAIN')
auc = roc_auc_score(y_train, y_pred_prob_2_train)
print("AUC: %1.3f" % auc)
print("GINI: %1.3f" % (2*auc-1))

print("/"*25, 'VALIDATION')
auc_2 = roc_auc_score(y_valid, y_pred_prob_2_valid)
print("AUC: %1.3f" % auc_2)
print("GINI: %1.3f" % (2*auc_2-1))

In [ ]:
print("/"*25, 'TRAIN')
print("RECALL     : %1.8f" % recall_score(y_train, y_pred_class_2_train))
print("EFECTIVIDAD: %1.8f" % precision_score(y_train, y_pred_class_2_train))
print("F1 SCORE   : %1.8f" % f1_score(y_train, y_pred_class_2_train))

print("/"*25, 'VALIDATION')
print("RECALL     : %1.8f" % recall_score(y_valid, y_pred_class_2_valid))
print("EFECTIVIDAD: %1.8f" % precision_score(y_valid, y_pred_class_2_valid))
print("F1 SCORE   : %1.8f" % f1_score(y_valid, y_pred_class_2_valid))

#### >>> RECALL

In [ ]:
from scipy.optimize import differential_evolution


optimization_rec = differential_evolution(
    lambda c: -1*recall_score(y_train, np.array([0 if _ < c[0] else 1 for _ in y_pred_prob_2_train])), 
    [(0, 1)]
)
optimization_rec

In [ ]:
corte_rec = optimization_rec['x'][0]
corte_rec

In [ ]:
y_pred_class_train_2_rec = np.where(y_pred_prob_2_train < corte_rec, 0 , 1)
y_pred_class_valid_2_rc = np.where(y_pred_prob_2_valid < corte_rec, 0 , 1)

In [ ]:
pd.Series(y_pred_class_train_2_rec).value_counts()

In [ ]:
pd.Series(y_pred_class_valid_2_rc).value_counts()

In [ ]:
print("/"*25, "TRAIN")
print("RECALL     : %1.8f" % recall_score(y_train, y_pred_class_train_2_rec))
print("EFECTIVIDAD: %1.8f" % precision_score(y_train, y_pred_class_train_2_rec))
print("F1 SCORE   : %1.8f" % f1_score(y_train, y_pred_class_train_2_rec))

print("/"*25, "VALIDATION")
print("RECALL     : %1.8f" % recall_score(y_valid, y_pred_class_valid_2_rc))
print("EFECTIVIDAD: %1.8f" % precision_score(y_valid, y_pred_class_valid_2_rc))
print("F1 SCORE   : %1.8f" % f1_score(y_valid, y_pred_class_valid_2_rc))

#### >>> EFECTIVIDAD

In [ ]:
optimization_pre = differential_evolution(
    lambda c: -1*precision_score(y_train, np.array([0 if _ < c[0] else 1 for _ in y_pred_prob_2_train])), 
    [(0, 1)]
)
optimization_pre

#### >>> AUC

In [ ]:
optimization_auc = differential_evolution(
    lambda c: -1*roc_auc_score(y_train, np.array([0 if _ < c[0] else 1 for _ in y_pred_prob_2_train])), 
    [(0, 1)]
)
optimization_auc

In [ ]:
corte_auc = optimization_auc['x'][0]
corte_auc

In [ ]:
y_pred_class_train_2_auc = np.where(y_pred_prob_2_train < corte_auc, 0 , 1)
y_pred_class_valid_2_auc = np.where(y_pred_prob_2_valid < corte_auc, 0 , 1)

In [ ]:
pd.Series(y_pred_class_train_2_auc).value_counts()

In [ ]:
pd.Series(y_pred_class_valid_2_auc).value_counts()

In [ ]:
print("/"*25, "TRAIN")
print("RECALL     : %1.8f" % recall_score(y_train, y_pred_class_train_2_auc))
print("EFECTIVIDAD: %1.8f" % precision_score(y_train, y_pred_class_train_2_auc))
print("F1 SCORE   : %1.8f" % f1_score(y_train, y_pred_class_train_2_auc))

print("/"*25, "VALIDATION")
print("RECALL     : %1.8f" % recall_score(y_valid, y_pred_class_valid_2_auc))
print("EFECTIVIDAD: %1.8f" % precision_score(y_valid, y_pred_class_valid_2_auc))
print("F1 SCORE   : %1.8f" % f1_score(y_valid, y_pred_class_valid_2_auc))